# Introduction to Web Crawlers 
In this tutorial, I'll be going over how to write a basic web crawler! ScrapeHero was a big help in compiling this tutorial! I would like to give them a special thanks all of their information and tutorials. All of tutorials used are linked in the references.

## What is a Web Crawler?
Before we start writing a web crawler, lets start by acutally defining what it is. A **web crawler**, also known as a **web spider**, is a program/script that when given a set of URLs, will traverse them in a recursive manner to gather more URLs to traverse. During this traversal their goal is typically to gather some sort of meta data regarding the pages they visit.

## Why Write a Web Crawler?
The internet as we know it would not function without our spider friends. Web cralwers are most famously known for their use in major search engines such as Google, Bing, Yahoo, etc. Search engines make use of web crawlers by deploying multiple spiders whose jobs are to maintain and update indexes. These indexes are used to give us these quick and relevant search options.

# Writing a Minimal Web Crawler
Given our definition of a web crawler, it seems simple enough, right? Lets just write a program that grabs all the 'a' tags from a url, puts them in some list, and just follows each URL to grab some more to tack onto our list! Unfortunately, things are never as simple as they seem. 

Imagine all of the different URL formats that we need to parse! We're going to be downloading all these URLs for something, how do we store that? How are we going to parse through all of the HTML to actually find something relevant to us? What happens when we have pages that are dynamically generated? What happens when there are relavant lists not listed in an 'a' tag? How do we gather links relevant to us? How do we exclude links we don't care about? How do we handle different errors or formatting that we've never encountered before? What are we hoping to achieve by creating this web crawler? How are we going to create a metric to rate the usefulness of a link? How can we do this in a way that won't degrade a website's performance? How do avoid getting blocked?

These are only some of the questions that one must think of when creating a web crawler. As we can see, writing a web crawler is more involved than it seems.


## Not all Spiders are Created Equal
Before really getting into crawling about the internet, lets talk about one of the important aspects of web scraping or crawling. One of the most important things you can do when scraping or crawling any websites, is to be courteous. **Being an ethical web scraper means to abide by the wishes of website owners and to respect others on the internet. This means to never bring any harm to a website!** 

Like actual spiders, web spiders are great when they stay in their lane and carry on their business out of sight. However,as soon as those spiders rapidly spread through your home, crawl over you and your possessions, and start their incessant biting, you'd probably go into a spider killing hysteria. This isn't too far off to how a sys admin would feel with some pesky and over eager spiders.

## How to be a friendly Spider
1. Follow the robots.txt of any website
2. Include your contact information in the UserAgent
3. Do not flood websites with your requests

Below I'll be showing you how to code your spider abide by all of these rules.

# Abiding by robots.txt
All websites have some set of rules they would like all bots to follow. This can be found for any website by accessing their robots.txt path. For examples, to find Google's robots.txt, we navigate to the URL https://google.com/robots.txt

Once here we can view which specific bots or what rules all bots should abide by. **Disallow** indicates to bots that they should not be following or downloading the paths listed.

Luckily the urllib library has a module to assist with this! This module will also assist in us setting a crawl rate for out spider to allow us to determine a good delay rate for a website.

In [1]:
from urllib.request import Request, urlopen
from urllib.error import URLError, HTTPError
from urllib.parse import urlparse
from bs4 import BeautifulSoup
#This is the module we'll use to go through robots.txt files
import urllib.robotparser

#Given a full and standard url, check if it is robot safe!
#We'll take care of standardizing them later when we parse through URLs
def isRobotSafe(givenURL):
    parsedURL = urlparse(givenURL)
    #Grab only the netloc and scheme to form the website's robots.txt full path
    robotURL = parsedURL.scheme + '://' + parsedURL.netloc + '/robots.txt'
    #print(robotURL)
    req = Request(robotURL)
    #Try to access the site, it it works, move one, if not catch the error
    try:
        response = urlopen(req)
    except URLError as e:
        if hasattr(e, 'reason'):
            print('We failed to reach a server.')
            print('Reason: ', e.reason)
            return False
        elif hasattr(e, 'code'):
            print('The server couldn\'t fulfill the request.')
            print('Error code: ', e.code)
            return False
    else:
        rp = urllib.robotparser.RobotFileParser()
        rp.set_url(robotURL)
        rp.read()
        print(givenURL)
        # The * indicates that all bots should abide by these rules
        return rp.can_fetch('*', givenURL)
        
print(isRobotSafe('https://www.imdb.com/showtimes/?ref_=nv_tp_sh_3'))
print(isRobotSafe('https://www.imdb.com/register'))

https://www.imdb.com/showtimes/?ref_=nv_tp_sh_3
True
https://www.imdb.com/register
False


# How to Prevent Getting Blacklisted from a Site
Even if you are a polite web scraper, there still exists websites that do not like spiders/bots and are against open access to data. In such cases, sys admins may immediately ban bot activity and have anti-spider detection on websites. In such cases, we may need to take some extra measures to not get blacklisted and look into the the anti-scraping practices of websites.

1. Rotate IPs and Proxy Services
2. User Agent Rotation and Spoofing
3. Make Spider Behaviour Look Less Robotic

# Rotating Proxies and IP Addresses
ScrapeHero has extensive tutorials on the subject of web scraping and one of their tutorials is about the subject title. Here we'll be using **https://free-proxy-list.net/** to get a list of proxies to use.

In [2]:
from urllib.request import Request, urlopen
from urllib.error import URLError, HTTPError
from urllib.parse import urlparse
from bs4 import BeautifulSoup
from itertools import cycle
import traceback
import random

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Safari/537.36'}

def getProxies():
    # This website has absolutely no restricted areas on their robots.txt!
    # So just be sure to be polite to them when we automate the scraping process!
    retProxyList = []
    url = 'https://free-proxy-list.net/'
    req = Request(url, headers = headers)
    try:
        response = urlopen(req)
    except URLError as e:
        if hasattr(e, 'reason'):
            print('Failed to access free-proxy-list.net!')
            print('Reason: ', e.reason)
            return retProxyList
        elif hasattr(e, 'code'):
            print('free-proxy-list.net couldn\'t fulfill the request!')
            print('Error code: ', e.code)
            return retProxyList
    else:
        bs = BeautifulSoup(response, 'html.parser')
        for row in bs.find("table", {"id":"proxylisttable"}).tbody.findAll('tr'):
            #print(row)
            anonLevel = row.findAll('td')[4].text
            if anonLevel == 'elite proxy':
                ip = row.findAll('td')[0].text
                port = row.findAll('td')[1].text
                retProxyList.append({'ip': ip, 'port': port})
                #print(anonLevel)
                #print(ip)
        return retProxyList


# We define a function to get a random index to test a proxy
def randomProxy(proxyList):
    randIndex = random.randint(0, len(proxyList) - 1)
    return randIndex

In [ ]:
#Simple Test of getProxies function
proxyList = getProxies()
url = 'https://www.httpbin.org/ip'
req = Request(url)
for proxyPair in proxyList:
    #print(proxyPair)
    fullProxy = proxyPair['ip'] + ':' + proxyPair['port']
    print(fullProxy)
    req.set_proxy(fullProxy, 'http')
    try:
        response = urlopen(req)
        print('Proxy success!')
    except:
        print('Error: proxy could not fulfill your request!')
            

185.146.112.253:8080
Proxy success!
1.179.164.233:58580
Error: proxy could not fulfill your request!
93.87.66.66:52761
Proxy success!
95.47.116.128:56792
Error: proxy could not fulfill your request!
190.152.12.54:41031
Proxy success!
118.174.220.148:54853
Error: proxy could not fulfill your request!
178.18.68.111:53281
Proxy success!
109.70.189.56:49061
Error: proxy could not fulfill your request!
46.146.213.163:42029
Error: proxy could not fulfill your request!
5.58.167.211:44952
Error: proxy could not fulfill your request!
177.23.104.38:44459
Error: proxy could not fulfill your request!
80.240.115.254:59458
Proxy success!
95.169.95.242:53803
Proxy success!
200.24.84.4:39136
Proxy success!
147.91.111.132:41032
Proxy success!
59.91.121.161:51445
Proxy success!
217.182.51.228:8080
Error: proxy could not fulfill your request!
103.205.26.84:41805
Error: proxy could not fulfill your request!
95.167.241.242:49636
Error: proxy could not fulfill your request!
197.232.16.104:44132
Proxy succes

In [ ]:
# OutLine for our WebCrawler Proxy Rotation
# We should next define some code that handles proxies that do not work, by deleting such proxies from the list
# We will continue doing so until we get a working proxy. Should the proxy list go empty, we should attempt to get more proxies
    
proxyList = getProxies()
randInd = randomProxy(proxyList)
maxPages = 20
rotation = 10

url = 'https://www.httpbin.org/ip'

for pagesTraversed in range(1, maxPages) :

    req = Request(url)
    proxyPair = proxyList[randInd]
    fullProxy = proxyPair['ip'] + ':' + proxyPair['port']
    print(fullProxy)
    req.set_proxy(fullProxy, 'http')
    try:
        response = urlopen(req)
    except:
        print('Error: proxy could not fulfill your request!')
        del proxyList[randInd]
        if len(proxyList) == 0:
            proxyList = getProxies()
    else:
        print('Proxy success!')
        pagesTraversed += 1
        if pagesTraversed % rotation == 0:
            randInd = randomProxy(proxyList)
            proxy = proxyList[randInd]

# Creating a User Agent for a Header
A user agent is a bit of text information that your browser will send to each website it makes a request to. Being a polite web crawler means to  include information like your contact information. Just something like the format below should suffice, but emailing a sysadmin beforehand is always nice. 

Typical User Agents look like this:

**headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Safari/537.36'}**

But for a simple and polite web scraper you might want to change this to include the following:

**headers = {
    "User-Agent": "my web scraping program. contact me at admin@domain.com"
}**

It's even better if you can provide a weblink of sorts where you include informaiton about your web crawler!

During your requests using urllid you would simply include this header like so:

**from urllib.request import Request
req = Request(url, headers = headers)**


In [ ]:
headers = {
    "User-Agent": "A beginner's web scraping program for demonstation. Contact me at rubui@ucsc.edu if there are any questions or concerns!"
}

# Faking and Rotating User Agents
Despite being a polite scraper, automated systems may still see large numbers of requests from the same IP and header and ban you. In such situations we can rotate our User Agents as well as IP to mask our spider's behavior.

If you'd like, it's possible to use your web scraping knowledge to gather a list of common and recently used User Agents, updating these every few day and rotate through these user agents in these requests. Below is a possible website to scrape:

https://developers.whatismybrowser.com/useragents/explore/

However, there are also third party packages that can handle this for us, such as the **fake-useragent** package. Its usage is detailed below.

In [ ]:
from fake_useragent import UserAgent
uagen = UserAgent()
print(uagen.chrome)
print(uagen.random)
print(uagen.random)

## All of Our Anti-Blacklist Methods Together

In [ ]:
from urllib.request import Request, urlopen
from urllib.error import URLError, HTTPError
from urllib.parse import urlparse
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
from itertools import cycle
import traceback
import random

uagen = UserAgent()
randUA = uagen.random
headers = {'User-Agent': randUA}

def getProxies():
    retProxyList = []
    url = 'https://free-proxy-list.net/'
    req = Request(url, headers = headers)
    try:
        response = urlopen(req)
    except URLError as e:
        if hasattr(e, 'reason'):
            print('Failed to access free-proxy-list.net!')
            print('Reason: ', e.reason)
            return retProxyList
        elif hasattr(e, 'code'):
            print('free-proxy-list.net couldn\'t fulfill the request!')
            print('Error code: ', e.code)
            return retProxyList
    else:
        bs = BeautifulSoup(response, 'html.parser')
        for row in bs.find("table", {"id":"proxylisttable"}).tbody.findAll('tr'):
            anonLevel = row.findAll('td')[4].text
            if anonLevel == 'elite proxy':
                ip = row.findAll('td')[0].text
                port = row.findAll('td')[1].text
                retProxyList.append({'ip': ip, 'port': port})
        return retProxyList

def randomProxy(proxyList):
    randIndex = random.randint(0, len(proxyList) - 1)
    return randIndex

proxyList = getProxies()
randInd = randomProxy(proxyList)

maxPages = 10
pagesTraversed = 0
rotation = 5
timeout = 20
randUA = uagen.random
headers = {'User-Agent': randUA}
url = 'https://www.httpbin.org/ip'

while pagesTraversed != maxPages :

    req = Request(url, headers)
    proxyPair = proxyList[randInd]
    fullProxy = proxyPair['ip'] + ':' + proxyPair['port']
    print(fullProxy)
    req.set_proxy(fullProxy, 'http')
    try:
        response = urlopen(url = req)
    except:
        print('Proxy Unresponsive')
        del proxyList[randInd]
        if len(proxyList) == 0:
            proxyList = getProxies()
            randInd = randomProxy(proxyList)
    
    else:
        bs = BeautifulSoup(response, 'html.parser')
        print(bs)
        pagesTraversed += 1
        if pagesTraversed % rotation == 0:
            #After a certain number of times a IP is used, rotate it and also change up the User Agent
            randInd = randomProxy(proxyList)
            proxy = proxyList[randInd]
            randUA = uagen.random
            headers = {'User-Agent': randUA}

# Writing Our Web Crawler
Now that we know how to be a polite web crawler and some measures to prevent getting blacklisted, lets finally start crawling!
**NOTE:** Despite spending all this time writing these anti-blacklist methods, in the following example, we will refrain from using them due to the amount of time needed to find an appropriate proxy.

# Pasring through Different URL formats 

(?:http://(?:(?:(?:(?:(?:[a-zA-Z\d](?:(?:[a-zA-Z\d]|-)*[a-zA-Z\d])?)\.
)*(?:[a-zA-Z](?:(?:[a-zA-Z\d]|-)*[a-zA-Z\d])?))|(?:(?:\d+)(?:\.(?:\d+)
){3}))(?::(?:\d+))?)(?:/(?:(?:(?:(?:[a-zA-Z\d$\-_.+!*'(),]|(?:%[a-fA-F
\d]{2}))|[;:@&=])*)(?:/(?:(?:(?:[a-zA-Z\d$\-_.+!*'(),]|(?:%[a-fA-F\d]{
2}))|[;:@&=])*))*)(?:\?(?:(?:(?:[a-zA-Z\d$\-_.+!*'(),]|(?:%[a-fA-F\d]{
2}))|[;:@&=])*))?)?)|(?:ftp://(?:(?:(?:(?:(?:[a-zA-Z\d$\-_.+!*'(),]|(?
:%[a-fA-F\d]{2}))|[;?&=])*)(?::(?:(?:(?:[a-zA-Z\d$\-_.+!*'(),]|(?:%[a-
fA-F\d]{2}))|[;?&=])*))?@)?(?:(?:(?:(?:(?:[a-zA-Z\d](?:(?:[a-zA-Z\d]|-
)*[a-zA-Z\d])?)\.)*(?:[a-zA-Z](?:(?:[a-zA-Z\d]|-)*[a-zA-Z\d])?))|(?:(?
:\d+)(?:\.(?:\d+)){3}))(?::(?:\d+))?))(?:/(?:(?:(?:(?:[a-zA-Z\d$\-_.+!
*'(),]|(?:%[a-fA-F\d]{2}))|[?:@&=])*)(?:/(?:(?:(?:[a-zA-Z\d$\-_.+!*'()
,]|(?:%[a-fA-F\d]{2}))|[?:@&=])*))*)(?:;type=[AIDaid])?)?)|(?:news:(?:
(?:(?:(?:[a-zA-Z\d$\-_.+!*'(),]|(?:%[a-fA-F\d]{2}))|[;/?:&=])+@(?:(?:(
?:(?:[a-zA-Z\d](?:(?:[a-zA-Z\d]|-)*[a-zA-Z\d])?)\.)*(?:[a-zA-Z](?:(?:[
a-zA-Z\d]|-)*[a-zA-Z\d])?))|(?:(?:\d+)(?:\.(?:\d+)){3})))|(?:[a-zA-Z](
?:[a-zA-Z\d]|[_.+-])*)|\*))|(?:nntp://(?:(?:(?:(?:(?:[a-zA-Z\d](?:(?:[
a-zA-Z\d]|-)*[a-zA-Z\d])?)\.)*(?:[a-zA-Z](?:(?:[a-zA-Z\d]|-)*[a-zA-Z\d
])?))|(?:(?:\d+)(?:\.(?:\d+)){3}))(?::(?:\d+))?)/(?:[a-zA-Z](?:[a-zA-Z
\d]|[_.+-])*)(?:/(?:\d+))?)|(?:telnet://(?:(?:(?:(?:(?:[a-zA-Z\d$\-_.+
!*'(),]|(?:%[a-fA-F\d]{2}))|[;?&=])*)(?::(?:(?:(?:[a-zA-Z\d$\-_.+!*'()
,]|(?:%[a-fA-F\d]{2}))|[;?&=])*))?@)?(?:(?:(?:(?:(?:[a-zA-Z\d](?:(?:[a
-zA-Z\d]|-)*[a-zA-Z\d])?)\.)*(?:[a-zA-Z](?:(?:[a-zA-Z\d]|-)*[a-zA-Z\d]
)?))|(?:(?:\d+)(?:\.(?:\d+)){3}))(?::(?:\d+))?))/?)|(?:gopher://(?:(?:
(?:(?:(?:[a-zA-Z\d](?:(?:[a-zA-Z\d]|-)*[a-zA-Z\d])?)\.)*(?:[a-zA-Z](?:
(?:[a-zA-Z\d]|-)*[a-zA-Z\d])?))|(?:(?:\d+)(?:\.(?:\d+)){3}))(?::(?:\d+
))?)(?:/(?:[a-zA-Z\d$\-_.+!*'(),;/?:@&=]|(?:%[a-fA-F\d]{2}))(?:(?:(?:[
a-zA-Z\d$\-_.+!*'(),;/?:@&=]|(?:%[a-fA-F\d]{2}))*)(?:%09(?:(?:(?:[a-zA
-Z\d$\-_.+!*'(),]|(?:%[a-fA-F\d]{2}))|[;:@&=])*)(?:%09(?:(?:[a-zA-Z\d$
\-_.+!*'(),;/?:@&=]|(?:%[a-fA-F\d]{2}))*))?)?)?)?)|(?:wais://(?:(?:(?:
(?:(?:[a-zA-Z\d](?:(?:[a-zA-Z\d]|-)*[a-zA-Z\d])?)\.)*(?:[a-zA-Z](?:(?:
[a-zA-Z\d]|-)*[a-zA-Z\d])?))|(?:(?:\d+)(?:\.(?:\d+)){3}))(?::(?:\d+))?
)/(?:(?:[a-zA-Z\d$\-_.+!*'(),]|(?:%[a-fA-F\d]{2}))*)(?:(?:/(?:(?:[a-zA
-Z\d$\-_.+!*'(),]|(?:%[a-fA-F\d]{2}))*)/(?:(?:[a-zA-Z\d$\-_.+!*'(),]|(
?:%[a-fA-F\d]{2}))*))|\?(?:(?:(?:[a-zA-Z\d$\-_.+!*'(),]|(?:%[a-fA-F\d]
{2}))|[;:@&=])*))?)|(?:mailto:(?:(?:[a-zA-Z\d$\-_.+!*'(),;/?:@&=]|(?:%
[a-fA-F\d]{2}))+))|(?:file://(?:(?:(?:(?:(?:[a-zA-Z\d](?:(?:[a-zA-Z\d]
|-)*[a-zA-Z\d])?)\.)*(?:[a-zA-Z](?:(?:[a-zA-Z\d]|-)*[a-zA-Z\d])?))|(?:
(?:\d+)(?:\.(?:\d+)){3}))|localhost)?/(?:(?:(?:(?:[a-zA-Z\d$\-_.+!*'()
,]|(?:%[a-fA-F\d]{2}))|[?:@&=])*)(?:/(?:(?:(?:[a-zA-Z\d$\-_.+!*'(),]|(
?:%[a-fA-F\d]{2}))|[?:@&=])*))*))|(?:prospero://(?:(?:(?:(?:(?:[a-zA-Z
\d](?:(?:[a-zA-Z\d]|-)*[a-zA-Z\d])?)\.)*(?:[a-zA-Z](?:(?:[a-zA-Z\d]|-)
*[a-zA-Z\d])?))|(?:(?:\d+)(?:\.(?:\d+)){3}))(?::(?:\d+))?)/(?:(?:(?:(?
:[a-zA-Z\d$\-_.+!*'(),]|(?:%[a-fA-F\d]{2}))|[?:@&=])*)(?:/(?:(?:(?:[a-
zA-Z\d$\-_.+!*'(),]|(?:%[a-fA-F\d]{2}))|[?:@&=])*))*)(?:(?:;(?:(?:(?:[
a-zA-Z\d$\-_.+!*'(),]|(?:%[a-fA-F\d]{2}))|[?:@&])*)=(?:(?:(?:[a-zA-Z\d
$\-_.+!*'(),]|(?:%[a-fA-F\d]{2}))|[?:@&])*)))*)|(?:ldap://(?:(?:(?:(?:
(?:(?:[a-zA-Z\d](?:(?:[a-zA-Z\d]|-)*[a-zA-Z\d])?)\.)*(?:[a-zA-Z](?:(?:
[a-zA-Z\d]|-)*[a-zA-Z\d])?))|(?:(?:\d+)(?:\.(?:\d+)){3}))(?::(?:\d+))?
))?/(?:(?:(?:(?:(?:(?:(?:[a-zA-Z\d]|%(?:3\d|[46][a-fA-F\d]|[57][Aa\d])
)|(?:%20))+|(?:OID|oid)\.(?:(?:\d+)(?:\.(?:\d+))*))(?:(?:%0[Aa])?(?:%2
0)*)=(?:(?:%0[Aa])?(?:%20)*))?(?:(?:[a-zA-Z\d$\-_.+!*'(),]|(?:%[a-fA-F
\d]{2}))*))(?:(?:(?:%0[Aa])?(?:%20)*)\+(?:(?:%0[Aa])?(?:%20)*)(?:(?:(?
:(?:(?:[a-zA-Z\d]|%(?:3\d|[46][a-fA-F\d]|[57][Aa\d]))|(?:%20))+|(?:OID
|oid)\.(?:(?:\d+)(?:\.(?:\d+))*))(?:(?:%0[Aa])?(?:%20)*)=(?:(?:%0[Aa])
?(?:%20)*))?(?:(?:[a-zA-Z\d$\-_.+!*'(),]|(?:%[a-fA-F\d]{2}))*)))*)(?:(
?:(?:(?:%0[Aa])?(?:%20)*)(?:[;,])(?:(?:%0[Aa])?(?:%20)*))(?:(?:(?:(?:(
?:(?:[a-zA-Z\d]|%(?:3\d|[46][a-fA-F\d]|[57][Aa\d]))|(?:%20))+|(?:OID|o
id)\.(?:(?:\d+)(?:\.(?:\d+))*))(?:(?:%0[Aa])?(?:%20)*)=(?:(?:%0[Aa])?(
?:%20)*))?(?:(?:[a-zA-Z\d$\-_.+!*'(),]|(?:%[a-fA-F\d]{2}))*))(?:(?:(?:
%0[Aa])?(?:%20)*)\+(?:(?:%0[Aa])?(?:%20)*)(?:(?:(?:(?:(?:[a-zA-Z\d]|%(
?:3\d|[46][a-fA-F\d]|[57][Aa\d]))|(?:%20))+|(?:OID|oid)\.(?:(?:\d+)(?:
\.(?:\d+))*))(?:(?:%0[Aa])?(?:%20)*)=(?:(?:%0[Aa])?(?:%20)*))?(?:(?:[a
-zA-Z\d$\-_.+!*'(),]|(?:%[a-fA-F\d]{2}))*)))*))*(?:(?:(?:%0[Aa])?(?:%2
0)*)(?:[;,])(?:(?:%0[Aa])?(?:%20)*))?)(?:\?(?:(?:(?:(?:[a-zA-Z\d$\-_.+
!*'(),]|(?:%[a-fA-F\d]{2}))+)(?:,(?:(?:[a-zA-Z\d$\-_.+!*'(),]|(?:%[a-f
A-F\d]{2}))+))*)?)(?:\?(?:base|one|sub)(?:\?(?:((?:[a-zA-Z\d$\-_.+!*'(
),;/?:@&=]|(?:%[a-fA-F\d]{2}))+)))?)?)?)|(?:(?:z39\.50[rs])://(?:(?:(?
:(?:(?:[a-zA-Z\d](?:(?:[a-zA-Z\d]|-)*[a-zA-Z\d])?)\.)*(?:[a-zA-Z](?:(?
:[a-zA-Z\d]|-)*[a-zA-Z\d])?))|(?:(?:\d+)(?:\.(?:\d+)){3}))(?::(?:\d+))
?)(?:/(?:(?:(?:[a-zA-Z\d$\-_.+!*'(),]|(?:%[a-fA-F\d]{2}))+)(?:\+(?:(?:
[a-zA-Z\d$\-_.+!*'(),]|(?:%[a-fA-F\d]{2}))+))*(?:\?(?:(?:[a-zA-Z\d$\-_
.+!*'(),]|(?:%[a-fA-F\d]{2}))+))?)?(?:;esn=(?:(?:[a-zA-Z\d$\-_.+!*'(),
]|(?:%[a-fA-F\d]{2}))+))?(?:;rs=(?:(?:[a-zA-Z\d$\-_.+!*'(),]|(?:%[a-fA
-F\d]{2}))+)(?:\+(?:(?:[a-zA-Z\d$\-_.+!*'(),]|(?:%[a-fA-F\d]{2}))+))*)
?))|(?:cid:(?:(?:(?:[a-zA-Z\d$\-_.+!*'(),]|(?:%[a-fA-F\d]{2}))|[;?:@&=
])*))|(?:mid:(?:(?:(?:[a-zA-Z\d$\-_.+!*'(),]|(?:%[a-fA-F\d]{2}))|[;?:@
&=])*)(?:/(?:(?:(?:[a-zA-Z\d$\-_.+!*'(),]|(?:%[a-fA-F\d]{2}))|[;?:@&=]
)*))?)|(?:vemmi://(?:(?:(?:(?:(?:[a-zA-Z\d](?:(?:[a-zA-Z\d]|-)*[a-zA-Z
\d])?)\.)*(?:[a-zA-Z](?:(?:[a-zA-Z\d]|-)*[a-zA-Z\d])?))|(?:(?:\d+)(?:\
.(?:\d+)){3}))(?::(?:\d+))?)(?:/(?:(?:(?:[a-zA-Z\d$\-_.+!*'(),]|(?:%[a
-fA-F\d]{2}))|[/?:@&=])*)(?:(?:;(?:(?:(?:[a-zA-Z\d$\-_.+!*'(),]|(?:%[a
-fA-F\d]{2}))|[/?:@&])*)=(?:(?:(?:[a-zA-Z\d$\-_.+!*'(),]|(?:%[a-fA-F\d
]{2}))|[/?:@&])*))*))?)|(?:imap://(?:(?:(?:(?:(?:(?:(?:[a-zA-Z\d$\-_.+
!*'(),]|(?:%[a-fA-F\d]{2}))|[&=~])+)(?:(?:;[Aa][Uu][Tt][Hh]=(?:\*|(?:(
?:(?:[a-zA-Z\d$\-_.+!*'(),]|(?:%[a-fA-F\d]{2}))|[&=~])+))))?)|(?:(?:;[
Aa][Uu][Tt][Hh]=(?:\*|(?:(?:(?:[a-zA-Z\d$\-_.+!*'(),]|(?:%[a-fA-F\d]{2
}))|[&=~])+)))(?:(?:(?:(?:[a-zA-Z\d$\-_.+!*'(),]|(?:%[a-fA-F\d]{2}))|[
&=~])+))?))@)?(?:(?:(?:(?:(?:[a-zA-Z\d](?:(?:[a-zA-Z\d]|-)*[a-zA-Z\d])
?)\.)*(?:[a-zA-Z](?:(?:[a-zA-Z\d]|-)*[a-zA-Z\d])?))|(?:(?:\d+)(?:\.(?:
\d+)){3}))(?::(?:\d+))?))/(?:(?:(?:(?:(?:(?:[a-zA-Z\d$\-_.+!*'(),]|(?:
%[a-fA-F\d]{2}))|[&=~:@/])+)?;[Tt][Yy][Pp][Ee]=(?:[Ll](?:[Ii][Ss][Tt]|
[Ss][Uu][Bb])))|(?:(?:(?:(?:[a-zA-Z\d$\-_.+!*'(),]|(?:%[a-fA-F\d]{2}))
|[&=~:@/])+)(?:\?(?:(?:(?:[a-zA-Z\d$\-_.+!*'(),]|(?:%[a-fA-F\d]{2}))|[
&=~:@/])+))?(?:(?:;[Uu][Ii][Dd][Vv][Aa][Ll][Ii][Dd][Ii][Tt][Yy]=(?:[1-
9]\d*)))?)|(?:(?:(?:(?:[a-zA-Z\d$\-_.+!*'(),]|(?:%[a-fA-F\d]{2}))|[&=~
:@/])+)(?:(?:;[Uu][Ii][Dd][Vv][Aa][Ll][Ii][Dd][Ii][Tt][Yy]=(?:[1-9]\d*
)))?(?:/;[Uu][Ii][Dd]=(?:[1-9]\d*))(?:(?:/;[Ss][Ee][Cc][Tt][Ii][Oo][Nn
]=(?:(?:(?:[a-zA-Z\d$\-_.+!*'(),]|(?:%[a-fA-F\d]{2}))|[&=~:@/])+)))?))
)?)|(?:nfs:(?:(?://(?:(?:(?:(?:(?:[a-zA-Z\d](?:(?:[a-zA-Z\d]|-)*[a-zA-
Z\d])?)\.)*(?:[a-zA-Z](?:(?:[a-zA-Z\d]|-)*[a-zA-Z\d])?))|(?:(?:\d+)(?:
\.(?:\d+)){3}))(?::(?:\d+))?)(?:(?:/(?:(?:(?:(?:(?:[a-zA-Z\d\$\-_.!~*'
(),])|(?:%[a-fA-F\d]{2})|[:@&=+])*)(?:/(?:(?:(?:[a-zA-Z\d\$\-_.!~*'(),
])|(?:%[a-fA-F\d]{2})|[:@&=+])*))*)?)))?)|(?:/(?:(?:(?:(?:(?:[a-zA-Z\d
\$\-_.!~*'(),])|(?:%[a-fA-F\d]{2})|[:@&=+])*)(?:/(?:(?:(?:[a-zA-Z\d\$\
-_.!~*'(),])|(?:%[a-fA-F\d]{2})|[:@&=+])*))*)?))|(?:(?:(?:(?:(?:[a-zA-
Z\d\$\-_.!~*'(),])|(?:%[a-fA-F\d]{2})|[:@&=+])*)(?:/(?:(?:(?:[a-zA-Z\d
\$\-_.!~*'(),])|(?:%[a-fA-F\d]{2})|[:@&=+])*))*)?)))


Source: https://stackoverflow.com/questions/827557/how-do-you-validate-a-url-with-a-regular-expression-in-python

Apparently the ReGex above is to validate any given URL. It's easy to see from this that one does not typically want to use regular expressions for this purpose. Luckily urlparse can assist us with our problem. Lets write some code how how to deal with different URL formats that we may have to deal with! We wish to get any URL in the format or a full path:

**scheme://netloc/path**

In [ ]:
# Initialize dictionary with list using defauldict 
# from collections import defaultdict 
from urllib.request import urlopen
from urllib.parse import urlparse
from bs4 import BeautifulSoup
import re
import random
  
# initializing dict with lists 
#pages = defaultdict(list)

#Here we only are taking care of it we are given correct paths or given full URLs
def parseHref(sourceUrl, givenHref):
    parsedHref = urlparse(givenHref)
    parsedSource = urlparse(sourceUrl)
    if(len(parsedHref.netloc) == 0):
        #If you're in this case, you were likely given a href/url that represents an internal link
        #So we should be appending onto it the current website
        retLink = parsedSource.scheme + '://' + parsedSource.netloc
        #If the fist think in the path does not have a '/' append one
        if(len(parsedHref.path) != 0):
            if(parsedHref.path[0] != '/'):
                retLink += '/'
            retLink += parsedHref.path
            return retLink
        else:
            return None
        #If we're here then we don't have a path or a netloc and may instead have an anchor, which we do not desire
        
    else:
        #Else if we are in this case, we are likely getting some sort of external link or full path name
        retLink = 'http://' + parsedHref.netloc + parsedHref.path
        return retLink

# Extracting All Links
Here we will extract links from a given web page. We will not be dealing with dynamic pages in this example. However, if you are interested in how one might do this, I would look into something like Selenium to deal with javascript and these pages.

In [ ]:
# This is a new function that we are creating to get links
# that our spider() function will call
def getLinks(startURL, linkList):
    # Remember the base URL which will be important when creating
    # absolute URLs 
    html = urlopen(startURL)
    bs = BeautifulSoup(html, 'html.parser')
    for link in bs.find_all('a'):
        if link.has_attr('href'):
            if link.attrs['href'] is not None:
                retLink = parseHref(startURL, link.attrs['href'])
                if(retLink is not None):
                    if retLink not in linkList:
                        linkList.append(retLink)

retLinks = []
getLinks('http://en.wikipedia.org', retLinks)

for l in retLinks:
    print(l)

## Extracting Only External Links or Internal Links
Be careful when using Regex especially when crawling or scraping the web. The non-uniformity can lead to unexpected consequences as shown below!

In [ ]:
#Retrieves a list of all unique Internal links found on a page
import re
def getInternalLinks(includeUrl, linkList):
    includeBase = '{}://{}'.format(urlparse(includeUrl).scheme, urlparse(includeUrl).netloc)
     
    html = urlopen(includeUrl)
    bs = BeautifulSoup(html, 'html.parser')
    #Finds all links that begin with a "/" or a link containing the base of the included link
    for link in bs.find_all('a', href=re.compile('^(/|.*'+includeBase+')')):
        #print(link)
        if link.attrs['href'] is not None:
            retLink = parseHref(includeUrl, link.attrs['href'])
            if(retLink is not None):
                if includeBase in retLink:
                    #Remove this line to show issue
                    if retLink not in linkList:
                        linkList.append(retLink)

internalList = []
getInternalLinks('http://en.wikipedia.org', internalList)
for l in internalList:
    print(l)         


In [ ]:
#Retrieves a list of all unique external links found on a page
def getExternalLinks(excludeUrl, linkList):
    # Remember the base URL which will be important when creating
    # absolute URLs 
    excludeBase = '{}://{}'.format(urlparse(excludeUrl).scheme, urlparse(excludeUrl).netloc)
    #print(excludeBase)
    html = urlopen(excludeUrl)
    bs = BeautifulSoup(html, 'html.parser')
    for link in bs.find_all('a'):
        if link.has_attr('href'):
            if link.attrs['href'] is not None:
                retLink = parseHref(excludeUrl, link.attrs['href'])
                if(retLink is not None):
                    if(excludeBase not in retLink):
                        if retLink not in linkList:
                            linkList.append(retLink)

retLinks = []
getExternalLinks('http://en.wikipedia.org', retLinks)

for l in retLinks:
    print(l)


# The Aimless Crawler
Here we put everything together to simply show how to gather links across the web. This does nothing besides traverse random external links of a starting website. The code below is from the book, with some minor changes on how internal and external websites are handled. The error in the notebook is due to a keyboard interupt which is needed, otherwise this would theoretically continue on forever.

In [ ]:
from urllib.request import Request, urlopen
from urllib.error import URLError, HTTPError
from urllib.parse import urlparse
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
from itertools import cycle

import traceback
import random
import http.client  # or http.client if you're on Python 3
http.client._MAXHEADERS = 1000

def getRandomExternalLink(startingPage):
    internalLinks = []
    externalLinks = []
    try:
        html = urlopen(startingPage)
    except URLError as e:
        if hasattr(e, 'reason'):
            print('Failed to access server.')
            print('Reason: ', e.reason)

        elif hasattr(e, 'code'):
            print('Website couldn\'t fulfill the request!')
            print('Error code: ', e.code)
        return None
    else:
        bs = BeautifulSoup(html, 'html.parser')
        getExternalLinks(startingPage, externalLinks)
        if len(externalLinks) == 0:
            print('No external links, looking around the site for one')
            domain = '{}://{}'.format(urlparse(startingPage).scheme, urlparse(startingPage).netloc)
            getInternalLinks(startingPage, internalLinks) 
            if(len(internalLinks) != 0):
                return getRandomExternalLink(internalLinks[random.randint(0, len(internalLinks)-1)])
            else:
                print("No internal links found")
                return
        else:
            return externalLinks[random.randint(0, len(externalLinks)-1)]
    
def followExternalOnly(startingSite):
    externalLink = getRandomExternalLink(startingSite)
    print('Random external link is: {}'.format(externalLink))
    if(externalLink is not None):
        followExternalOnly(externalLink)
    else:
        print('Encountered an "end" to the chain')

followExternalOnly('http://www.oreilly.com')

In [ ]:
# Collects a list of all external URLs found on the site
allExtLinks = []
allIntLinks = []

def getAllExternalLinks(siteUrl):   
    try:
        html = urlopen(siteUrl)
    except URLError as e:
        if hasattr(e, 'reason'):
            print('Failed to access server.')
            print('Reason: ', e.reason)

        elif hasattr(e, 'code'):
            print('Website couldn\'t fulfill the request!')
            print('Error code: ', e.code)
        return None
    else:
        internalLinks = []
        externalLinks = []
        bs = BeautifulSoup(html, 'html.parser')
        getInternalLinks(siteUrl, internalLinks)
        getExternalLinks(siteUrl, externalLinks)

        for link in externalLinks:
            if link not in allExtLinks:
                allExtLinks.append(link)
                print(link)
        for link in internalLinks:
            if link not in allIntLinks:
                allIntLinks.append(link)
                getAllExternalLinks(link)


allIntLinks.append('http://www.oreilly.com')
getAllExternalLinks('http://www.oreilly.com')

# References
ScrapeHero was a big help in this tutorial, special thanks to them for all of their information and tutorials

### General
1. Web Scraping with Python, Chapter 3 by Ryan Mitchell
2. Writing a Polite Web Scraper:
https://blog.scrapinghub.com/2016/08/25/how-to-crawl-the-web-politely-with-scrapy
3. Sample Java Code for a Web Crawler: 
https://cs.nyu.edu/courses/spring16/CSCI-GA.2580-001/WebCrawler.java.txt
4. Make your Own Web Crawler Tutorial in PHP:
https://www.youtube.com/watch?v=_05gKTbN6RA
5. Description of a Web Crawler:
https://stackoverflow.com/questions/4278024/a-very-simple-c-web-crawler-spider
6. Handling URLErrors, HTTPErrors, and User Agents
https://docs.python.org/3/howto/urllib2.html
7. Webscraping Cheat Sheet:
https://blog.hartleybrody.com/web-scraping-cheat-sheet/
8. How to Prevent Getting Blacklisted while Scraping:
https://www.scrapehero.com/how-to-prevent-getting-blacklisted-while-scraping/
9. Robot Parser Documentation:
https://docs.python.org/3/library/urllib.robotparser.html

### Proxy Rotation
1. How to Rotate Proxies and IP Addresses using Python 3: https://www.scrapehero.com/how-to-rotate-proxies-and-ip-addresses-using-python-3/
2. Creating a Crawler with a Rotating IP:
https://codelike.pro/create-a-crawler-with-rotating-ip-proxy-in-python/
3. Creating a Reliable Proxy Method:
https://www.youtube.com/watch?v=n3uSyqoBgQI

### User Agent Rotation
1. How to Fake and Rotate User Agents:
https://www.scrapehero.com/how-to-fake-and-rotate-user-agents-using-python-3/
2. Fake-UserAgent Package: 
https://pypi.org/project/fake-useragent/




# Used Packages:
1. pip install fake-useragent